Not used.

In [21]:
models = joblib.load('temp/a1/2/xgb.pkl')

In [22]:
models

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing...traints=None,
                                     n_estimators=100, n_jobs=6,
                                     num_parallel_tree=None, random_state=1,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                    

In [ ]:
models = joblib.load('temp/a1/xgb_hist_maxdepth1.pkl')

df = pd.read_pickle('temp/a1/features.pkl')
X = df.drop(columns='target').values
y = df['target']

outer_cv = StratifiedKFold(5, shuffle=True, random_state=1).split(X, y)
probas_preds = list()
y_trues = list()
aps = list()
for clf, (train_index, test_index) in zip(models['estimator'], outer_cv):
  probas_pred = clf.best_estimator_.predict_proba(X[test_index])[:, 1]
  y_true = y[test_index].values
  
  ap = average_precision_score(y_true, y_score=probas_pred)
  
  probas_preds.append(probas_pred)
  y_trues.append(y_true)
  aps.append(ap)
probas_pred = np.array(probas_preds).flatten()
y_true = np.array(y_trues).flatten()
ap = aps

In [ ]:
print(f'average precision: {np.mean(ap):.4f} +- {np.std(ap):.4f}')

In [ ]:
fig = ff.create_distplot(
  [probas_pred[y_true], np.random.choice(probas_pred[~y_true], 10000)], 
  group_labels=['positives', 'negatives'], 
  bin_size=.01,
  colors=['green', 'red'],
)
fig.update_xaxes(range=[0,1])
fig.update_layout(xaxis_title='score')

In [ ]:
%%time
p, r, t = precision_recall_curve(y_true, probas_pred)
data = pd.DataFrame(
  dict(precision=p, recall=r, thresholds=np.append(t, [1]))
)[::1000]

In [ ]:
%%time
fig = px.scatter(
  data, 
  x='recall', 
  y='precision', 
  title='Precision vs recall', 
  range_x=(0,1), 
  range_y=(0,1), 
  width=400, 
  height=400
)
fig.show()
print(average_precision_score(y_true, y_score=probas_pred))

In [ ]:
fig = go.Figure(
  go.Scatter(
    x=data['recall'],
    y=data['precision'],
    mode='lines+markers',
    marker=dict(cmin=0, cmax=1, color=data['thresholds'], colorscale='Viridis', colorbar=dict(title="Threshold")),
    text=data['thresholds'],
    fill='tozeroy'
  )
)
fig.update_layout(
  xaxis_title='recall', 
  yaxis_title='precision', 
  title='Precision vs recall'
)
fig.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_score=probas_pred)
n = sum(~y_true)
p = sum(y_true)
data = pd.DataFrame(dict(fp=fpr*n, tp=tpr*p, thresholds=thresholds))[::10000]

In [ ]:
%%time
fig = go.Figure(
  go.Scatter(
    x=data['fp'],
    y=data['tp'],
    mode='lines+markers',
    marker=dict(cmin=0, cmax=1, color=data['thresholds'], colorscale='Viridis', colorbar=dict(title="Threshold")),
    text=data['thresholds'],
    fill='tozeroy'
  )
)
fig.update_layout(
  title='ROC', xaxis_title='true positives', yaxis_title='false positives'
)
fig.show()
print(f'{auc(fpr, tpr):.2f}')